# Entregable: Preprocesamiento de datos multivariados o de una serie temporal

This is business case prepared for the Statistics Module (Bloque 1) of the Advanced AI concentration



## Case Description

Tienes que utilizar las variables explicativas y dependientes de tu ÚLTIMO modelo que hiciste en el entregable del Bloque 1. Recuerda que las variables independientes deben ser razones financieras así como las variables dummy (0/1) de la variable de tamaño ("size").

1. Realiza un análisis exploratorio de las variables:

a) Calcula matriz de varianza y covarianza, así como matriz de correlación de las variables independientes y la dependiente. Explicar qué es la varianza, covarianza y correlación. Interpreta la matriz de correlación. Tiene que utilizar álgebra matricial y corroborar resultados con funciones de Python.

b) Corre pruebas estadísticas para detectar outliers y leverage points. Tiene que utilizar álgebra matricial para las pruebas y explicar claramente cómo funcionan las pruebas. Puede utilizar funciones de Python para corroborar resultados.

2. Hace un análisis de multicolinealidad explicando la prueba e implicaciones en el modelo.
3. Propone e implementa soluciones a los problemas de los puntos anteriores para que el modelo sea el más adecuado.
4. Estima e interpreta un modelo de regresión múltiple después de atender los problemas anteriores. Tiene que utilizar álgebra matricial para estimar coeficientes y errores estándar del modelo de regresión, y utilizar funciones de Python para corroborar resultados.

## Carga y Seleccion de Datos

In [2]:
from google.colab import drive
import plotly.express as px
import pandas as pd
import numpy as np
drive.mount("/content/gdrive")  

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
%cd "/content/gdrive/MyDrive/IA/Estadistica"
!ls

/content/gdrive/MyDrive/IA/Estadistica
BusinessCase.ipynb  Entregable1.ipynb  us2022q2a.csv  usfirms2022.csv


In [4]:
df1 = pd.read_csv('us2022q2a.csv')
df2 = pd.read_csv('usfirms2022.csv')

## Limpieza de datos

In [5]:
df1 = df1.drop(['fiscalmonth', 'year', 'cto'], axis=1)
df2 = df2.drop(['N', 'Country\nof Origin', 'Type of Asset'], axis=1)

In [6]:
df_merge = df1.merge(df2, left_on='firm', right_on='Ticker')
df_merge

,firm,q,revenue,cogs,sgae,otheropexp,extraincome,finexp,incometax,totalassets,...,originalprice,sharesoutstanding,Ticker,Name,Class,Sector NAICS\nlevel 1,Exchange / Src,Sector\nEconomatica,Sector NAICS\nlast available,partind
0,A,2000q1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,104.0000,452000.000,A,"Agilent Technologies, Inc",Com,Manufacturing,NYSE,Electric Electron,"Navigational, Measuring, Electromedical, and C...",0.124
1,A,2000q2,2485000.0,1261000.0,1.010000e+06,0.0,42000.000000,0.000,90000.0,7321000.000,...,73.7500,452271.967,A,"Agilent Technologies, Inc",Com,Manufacturing,NYSE,Electric Electron,"Navigational, Measuring, Electromedical, and C...",0.124
2,A,2000q3,2670000.0,1369000.0,1.091000e+06,0.0,28000.000000,0.000,83000.0,7827000.000,...,48.9375,453014.579,A,"Agilent Technologies, Inc",Com,Manufacturing,NYSE,Electric Electron,"Navigational, Measuring, Electromedical, and C...",0.124
3,A,2000q4,3372000.0,1732000.0,1.182000e+06,0.0,10000.000000,0.000,163000.0,8425000.000,...,54.7500,456366.381,A,"Agilent Technologies, Inc",Com,Manufacturing,NYSE,Electric Electron,"Navigational, Measuring, Electromedical, and C...",0.124
4,A,2001q1,2841000.0,1449000.0,1.113000e+06,0.0,-6000.000000,0.000,119000.0,9208000.000,...,30.7300,456769.737,A,"Agilent Technologies, Inc",Com,Manufacturing,NYSE,Electric Electron,"Navigational, Measuring, Electromedical, and C...",0.124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323811,ZYNE,2021q2,0.0,0.0,9.838494e+03,0.0,-117.528220,-5.943,0.0,98195.904,...,5.2900,41251.537,ZYNE,"Zynerba Pharmaceuticals, Inc",Com,Manufacturing,NASDAQ,Chemical,Pharmaceutical and Medicine Manufacturing,-
323812,ZYNE,2021q3,0.0,0.0,1.021065e+04,0.0,-376.636750,-5.038,0.0,89996.170,...,4.2400,41251.537,ZYNE,"Zynerba Pharmaceuticals, Inc",Com,Manufacturing,NASDAQ,Chemical,Pharmaceutical and Medicine Manufacturing,-
323813,ZYNE,2021q4,0.0,0.0,8.836436e+03,0.0,16.937906,-4.433,0.0,81171.507,...,2.8800,41217.537,ZYNE,"Zynerba Pharmaceuticals, Inc",Com,Manufacturing,NASDAQ,Chemical,Pharmaceutical and Medicine Manufacturing,-
323814,ZYNE,2022q1,0.0,0.0,8.903915e+03,0.0,317.252110,-96.044,0.0,74381.029,...,2.0500,42447.037,ZYNE,"Zynerba Pharmaceuticals, Inc",Com,Manufacturing,NASDAQ,Chemical,Pharmaceutical and Medicine Manufacturing,-


In [7]:
df2_services = df_merge.copy()
df2_services = df2_services.drop(df2_services[(df2_services['Sector NAICS\nlevel 1'] == 'Manufacturing')].index)
df2_services = df2_services.drop(df2_services[(df2_services['Sector NAICS\nlevel 1'] == 'Finance and Insurance')].index)
df2_services = df2_services.drop(df2_services[(df2_services['Sector NAICS\nlevel 1'] == 'Information')].index)
df2_services = df2_services.drop(df2_services[(df2_services['Sector NAICS\nlevel 1'] == 'Retail Trade')].index)
df2_services = df2_services.drop(df2_services[(df2_services['Sector NAICS\nlevel 1'] == 'Mining, Quarrying, and Oil and Gas Extraction')].index)
df2_services = df2_services.drop(df2_services[(df2_services['Sector NAICS\nlevel 1'] == 'Wholesale Trade')].index)
df2_services = df2_services.drop(df2_services[(df2_services['Sector NAICS\nlevel 1'] == 'Utilities')].index)
df2_services = df2_services.drop(df2_services[(df2_services['Sector NAICS\nlevel 1'] == 'Transportation and Warehousing')].index)
df2_services = df2_services.drop(df2_services[(df2_services['Sector NAICS\nlevel 1'] == 'Real Estate and Rental and Leasing')].index)
df2_services = df2_services.drop(df2_services[(df2_services['Sector NAICS\nlevel 1'] == 'Health Care and Social Assistance')].index)
df2_services = df2_services.drop(df2_services[(df2_services['Sector NAICS\nlevel 1'] == 'Construction')].index)
df2_services = df2_services.drop(df2_services[(df2_services['Sector NAICS\nlevel 1'] == 'Arts, Entertainment, and Recreation')].index)
df2_services = df2_services.drop(df2_services[(df2_services['Sector NAICS\nlevel 1'] == '-')].index)
df2_services = df2_services.drop(df2_services[(df2_services['Sector NAICS\nlevel 1'] == 'Agriculture, Forestry, Fishing and Hunting')].index)

In [8]:
df2_services['R'] = np.log(df2_services.groupby(['firm'])['adjprice'].shift(-1)) - np.log(df2_services.groupby(['firm'])['adjprice'].shift(3))

In [9]:
df_mask = df2_services['q'] == '2022q1'
df3 = df2_services[df_mask]

In [10]:
df3 = df3.drop(
    [
        'extraincome', 
        'shortdebt', 
        'longdebt', 
        'stockholderequity', 
        'Ticker', 
        'Name', 
        'Class',
        'Exchange / Src',
        'Sector\nEconomatica',
        'Sector NAICS\nlast available',
        'partind'
      ],
       axis=1)

In [11]:
df3['revenue'].describe()

count    3.720000e+02
mean     9.212180e+05
std      3.942143e+06
min      0.000000e+00
25%      6.649075e+04
50%      1.981825e+05
75%      7.438012e+05
max      6.801100e+07
Name: revenue, dtype: float64

In [12]:
df_mask = df3['R'] > 0
df3 = df3[df_mask]

In [13]:
df3['Book'] = df3['totalassets'] - df3['totalliabilities']
df3['Market'] = df3['originalprice'] * df3['sharesoutstanding']


df3['EBIT'] = df3['revenue'] - df3['cogs'] - df3['sgae'] - df3['otheropexp']
df3['NetIncome'] = df3['EBIT'] - df3['incometax'] - df3['finexp']
df3['EPS'] = df3['NetIncome'] / df3['sharesoutstanding']


df3['EPSP'] = df3['EPS'] / df3['originalprice']
df3['OPM'] = df3['EBIT'] / df3['revenue']
df3['Book_to_Market_ratio'] = df3['Book'] / df3['Market']

In [14]:
df3['R'].describe()

count    64.000000
mean      0.194677
std       0.223939
min       0.003957
25%       0.071489
50%       0.133235
75%       0.269808
max       1.591808
Name: R, dtype: float64

In [15]:
df_mask = df3['R'] >= -5
df_clean = df3[df_mask]

In [16]:
df_clean['EPSP'].describe()

count    64.000000
mean      0.012926
std       0.023902
min      -0.029480
25%       0.003823
50%       0.008263
75%       0.015408
max       0.157480
Name: EPSP, dtype: float64

In [17]:
df_mask = df_clean['EPSP'] >= -5
df_clean_2 = df_clean[df_mask]

In [18]:
df_clean_2['Book_to_Market_ratio'].describe()

count    64.000000
mean      0.302797
std       0.250904
min      -0.247891
25%       0.108701
50%       0.254069
75%       0.425732
max       1.119752
Name: Book_to_Market_ratio, dtype: float64

In [19]:
df_mask = df_clean_2['Book_to_Market_ratio'] >= -5
df_clean_3 = df_clean_2[df_mask]

In [20]:
df_clean_2['OPM'].describe()

count    64.000000
mean      0.130387
std       0.122849
min      -0.107435
25%       0.046636
50%       0.099495
75%       0.188092
max       0.441066
Name: OPM, dtype: float64

In [21]:
df_clean_3 = df_clean_3.sort_values(by = 'Market')

In [22]:
df_clean_3 = df_clean_3.reset_index()
df_clean_3 = df_clean_3.drop(['index'], axis = 1)

In [ ]:
df_clean_3 = df_clean_3.reset_index()

In [23]:
df_clean_3["isSmall"] = df_clean_3.index <= len(df_clean_3) / 3
df_clean_3["isSmall"] = df_clean_3["isSmall"].astype(int)

df_clean_3["isMedium"] = (df_clean_3.index <= (2 * len(df_clean_3) / 3)) & (df_clean_3.index > (len(df_clean_3) / 3))
df_clean_3["isMedium"] = df_clean_3["isMedium"].astype(int)

In [24]:
Variables = df_clean_3[['EPSP', 'OPM', 'Book_to_Market_ratio', 'isSmall', 'isMedium', 'R']]
Variables_mat = Variables.cov().to_numpy()

In [25]:
X = df_clean_3[['EPSP', 'OPM', 'Book_to_Market_ratio', 'isSmall', 'isMedium']]
y = df_clean_3['R']

In [26]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns
  
# calculating VIF for each feature
vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                          for i in range(len(X.columns))]

print(vif_data)

                feature       VIF
0                  EPSP  1.624803
1                   OPM  1.655611
2  Book_to_Market_ratio  2.886763
3               isSmall  2.256689
4              isMedium  2.096322


##Matriz de varianza y covarianza

Seguimos las formula del documento https://docs.google.com/viewer?a=v&pid=sites&srcid=ZWdhZGVyemMubmV0fGZ6MzAzMHxneDozOTI1ODBmMGU2YTMzZGM1

Para obtener la matriz de varianza y covarianza

In [27]:
X_varcov_mat = X.to_numpy()

In [28]:
X_varcov_trans = np.transpose(X_varcov_mat)

In [29]:
aux1 = np.dot(X_varcov_trans, X_varcov_mat)

In [30]:
unos = np.zeros((X_varcov_trans.shape[1], 1))

In [31]:
unos = 1 + unos

In [32]:
aux2 = np.dot(X_varcov_trans, unos)

In [33]:
aux3 = np.transpose(aux2)

In [34]:
aux4 = np.dot(aux2, aux3)

In [35]:
aux5 = aux4 / X_varcov_mat.shape[0]

In [36]:
aux6 = aux1 - aux5

In [37]:
mat_varcov = aux6 / (X_varcov_mat.shape[0] - 1)

In [38]:
mat_varcov

array([[ 0.00057129,  0.00110538, -0.00087653,  0.0005751 ,  0.000365  ],
       [ 0.00110538,  0.01509194, -0.01598234, -0.02142199, -0.00162864],
       [-0.00087653, -0.01598234,  0.06295262,  0.03482429,  0.02106378],
       [ 0.0005751 , -0.02142199,  0.03482429,  0.22916667, -0.11458333],
       [ 0.000365  , -0.00162864,  0.02106378, -0.11458333,  0.22395833]])

In [39]:
mat = pd.DataFrame(mat_varcov)

In [40]:
mat

,0,1,2,3,4
0,0.000571,0.001105,-0.000877,0.000575,0.000365
1,0.001105,0.015092,-0.015982,-0.021422,-0.001629
2,-0.000877,-0.015982,0.062953,0.034824,0.021064
3,0.000575,-0.021422,0.034824,0.229167,-0.114583
4,0.000365,-0.001629,0.021064,-0.114583,0.223958


In [41]:
X.cov()

,EPSP,OPM,Book_to_Market_ratio,isSmall,isMedium
EPSP,0.000571,0.001105,-0.000877,0.000575,0.000365
OPM,0.001105,0.015092,-0.015982,-0.021422,-0.001629
Book_to_Market_ratio,-0.000877,-0.015982,0.062953,0.034824,0.021064
isSmall,0.000575,-0.021422,0.034824,0.229167,-0.114583
isMedium,0.000365,-0.001629,0.021064,-0.114583,0.223958


Comprobamos que las varianzas son iguales en nuestro calculo y lo obtenido por medio de pandas, por lo que la matriz es correcta

##Matriz de correlaciones


Seguimos la formula de correlacion obtenida a base de los standar deviations y la matriz de varianza y covarianza

In [42]:
var = np.diagonal(mat_varcov)

In [43]:
stdr_dev = np.sqrt(var)

In [44]:
stdr_dev

array([0.02390169, 0.12284925, 0.2509036 , 0.47871355, 0.47324236])

In [45]:
ya_no_se = [[ 0 for y in range( len(stdr_dev) ) ] for x in range( len(stdr_dev) )]
for i in range(len(stdr_dev)):
  for j in range(len(stdr_dev)):
    ya_no_se[i][j] = stdr_dev[i] * stdr_dev[j]

In [46]:
corr = [[ 0 for y in range( len(ya_no_se) ) ] for x in range( len(ya_no_se) )]
for i in range(len(ya_no_se)):
  for j in range(len(ya_no_se)):
    corr[i][j] = mat_varcov[i][j] / ya_no_se[i][j]

In [47]:
corr

[[0.9999999999999998,
  0.3764513838918727,
  -0.14616089228811677,
  0.0502615792665872,
  0.032268836110310326],
 [0.3764513838918727,
  0.9999999999999999,
  -0.5185143922484545,
  -0.3642600879871348,
  -0.028013613157682502],
 [-0.14616089228811677,
  -0.5185143922484545,
  1.0000000000000002,
  0.2899343236215871,
  0.1773967997169645],
 [0.0502615792665872,
  -0.3642600879871348,
  0.2899343236215871,
  1.0,
  -0.5057805388588731],
 [0.032268836110310326,
  -0.028013613157682502,
  0.1773967997169645,
  -0.5057805388588731,
  1.0]]

In [48]:
X.corr()

,EPSP,OPM,Book_to_Market_ratio,isSmall,isMedium
EPSP,1.000000,0.376451,-0.146161,0.050262,0.032269
OPM,0.376451,1.000000,-0.518514,-0.364260,-0.028014
Book_to_Market_ratio,-0.146161,-0.518514,1.000000,0.289934,0.177397
isSmall,0.050262,-0.364260,0.289934,1.000000,-0.505781
isMedium,0.032269,-0.028014,0.177397,-0.505781,1.000000


Comprobamos que las correlaciones son iguales en nuestro calculo y lo obtenido por medio de pandas, por lo que la matriz es correcta